<a href="https://colab.research.google.com/github/faezesarlakifar/ChillinWars-Agent/blob/main/GeneticMiniMax_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import numpy as np
import copy

# chillin imports
from chillin_client import RealtimeAI

# project imports
from ks.models import ECell, EDirection, Position
from ks.commands import ChangeDirection, ActivateWallBreaker

# Constants
POPULATION_SIZE = 10
MAX_GENERATIONS = 10
MAX_DEPTH = 3
MUTATION_RATE = 0.1


class AI(RealtimeAI):

    def __init__(self, world):
        super(AI, self).__init__(world)
        self.world = world

    def initialize(self):
        print('initialize')

    def decide(self):
        print('decide')
        self.client1()

    def client1(self):
        state = self.world  # Initial state
        my_team = self.my_side

        best_direction = self.genetic_minimax(state, MAX_GENERATIONS, POPULATION_SIZE)
        self.send_command(ChangeDirection(best_direction))

    def genetic_minimax(self, state, max_generations, population_size):
        population = self.initialize_population(population_size)
        for generation in range(max_generations):
            fitness_scores = []
            for individual in population:
                fitness_score = self.evaluate_individual(state, individual)
                fitness_scores.append(fitness_score)

            population = self.select_parents(population, fitness_scores, population_size)
            population = self.crossover(population, population_size)
            population = self.mutate(population)

        best_individual = population[0]
        return best_individual

      def initialize_population(self, population_size):
      population = []
      directions = list(EDirection)
      for _ in range(population_size):
          individual = [random.choice(directions)]
          for _ in range(MAX_DEPTH - 1):
              previous_direction = individual[-1]
              valid_directions = [direction for direction in directions if not self.is_opposite(previous_direction, direction)]
              individual.append(random.choice(valid_directions))
          population.append(individual)
      return population

    def mutate(self, population):
        for i in range(len(population)):
            for j in range(MAX_DEPTH):
                if random.random() < MUTATION_RATE:
                    previous_direction = population[i][j-1]
                    valid_directions = [direction for direction in list(EDirection) if not self.is_opposite(previous_direction, direction)]
                    population[i][j] = random.choice(valid_directions)
        return population

    def is_opposite(self, direction1, direction2):
        if (direction1 == EDirection.UP and direction2 == EDirection.DOWN) or \
                (direction1 == EDirection.DOWN and direction2 == EDirection.UP) or \
                (direction1 == EDirection.LEFT and direction2 == EDirection.RIGHT) or \
                (direction1 == EDirection.RIGHT and direction2 == EDirection.LEFT):
            return True
        return False

    def evaluate_individual(self, state, individual):
        total_score = 0
        current_state = copy.deepcopy(state)
        for direction in individual:
            next_state = self.get_next_state(current_state, direction)
            total_score += self.evaluate_state(next_state)
            current_state = next_state
        return total_score

    def select_parents(self, population, fitness_scores, population_size):
        selected_population = []
        probabilities = np.array(fitness_scores) / sum(fitness_scores)
        for _ in range(population_size):
            selected_individual = random.choices(population, probabilities)[0]
            selected_population.append(selected_individual)
        return selected_population

    def crossover(self, population, population_size):
        new_population = []
        for _ in range(population_size):
            parent1 = random.choice(population)
            parent2 = random.choice(population)
            crossover_point = random.randint(1, MAX_DEPTH - 1)
            child = parent1[:crossover_point] + parent2[crossover_point:]
            new_population.append(child)
        return new_population

    def evaluate_state(self, state):
        #TODO
        return state_score

    def get_next_state(self, state, direction):
        #TODO
        return next_state
